In [49]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import glob
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import stats as st
from scipy.stats import norm
import statsmodels.formula.api as sm
import datetime
import re
import shutil
import string
from __future__ import division, print_function

now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d_%H:%M")
month = now.strftime("%B")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
import sys
%matplotlib inline

## Importing Data From Each Trial, For Each Participant

In [50]:
# load the csv files into pandas
sleep_df = pd.DataFrame()

# starting dir
start_dir = os.getcwd() + "/data/"

# Get a list of dirs
participant_dirs = os.listdir( start_dir )

# for each folder in the glob
for participant in participant_dirs:
    # sanity check: make sure its a directory
    participant_path = start_dir + participant
    if os.path.isdir(participant_path):
        # for each csv in the folder
        part_glob = glob.glob( participant_path + "/*.csv" )

        # add to the table
        for csv in part_glob:
            df = pd.read_csv(csv)
            sleep_df = sleep_df.append(df)


# check to make sure that csv content is added to data frame
sleep_df.head()


,Unnamed: 12,Unnamed: 13,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 5,cat.thisIndex,cat.thisN,cat.thisRepN,cat.thisTrialN,correct,date,expName,frameRate,key_resp_2.keys,key_resp_2.rt,key_resp_3.keys,key_resp_3.rt,key_resp_7.keys,key_resp_7.rt,key_resp_8.keys,key_resp_8.rt,key_resp_9.corr,key_resp_9.keys,key_resp_9.rt,participant,practice.thisIndex,practice.thisN,practice.thisRepN,practice.thisTrialN,session,stim1,stim2,ws.thisIndex,ws.thisN,ws.thisRepN,ws.thisTrialN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2015_Jun_13_1126,Stat_Cat_Oct,59.4568,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,6.90258,1.0,0.0,0.0,0.0,0.0,1.0,prac1.wav,prac2.wav,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015_Jun_13_1126,Stat_Cat_Oct,59.4568,NaN,NaN,space,14.678775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2015_Jun_13_1126,Stat_Cat_Oct,59.4568,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,0.558629,1.0,NaN,NaN,NaN,NaN,1.0,ws/19.wav,ws/1.wav,6.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2015_Jun_13_1126,Stat_Cat_Oct,59.4568,['2'],[2.8283094843209255],NaN,NaN,NaN,NaN,NaN,NaN,1.0,2,1.62566,1.0,NaN,NaN,NaN,NaN,1.0,ws/22.wav,ws/4.wav,10.0,1.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2015_Jun_13_1126,Stat_Cat_Oct,59.4568,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,0.117813,1.0,NaN,NaN,NaN,NaN,1.0,ws/1.wav,ws/10.wav,5.0,2.0,0.0,2.0


In [51]:
# get a list of all columns
cols = list(sleep_df)

# create a table with *only* the columns we want
sleep_columns = ['participant','stim1','stim2','correct','key_resp_2.keys','key_resp_9.keys','key_resp_9.corr','key_resp_9.rt','session']
sleep_df = sleep_df[sleep_columns]

print(start_dir)

# sanity chek
sleep_df.head()


C:\Users\Ashkan\Dropbox\sleep_stat_cat/data/


,participant,stim1,stim2,correct,key_resp_2.keys,key_resp_9.keys,key_resp_9.corr,key_resp_9.rt,session
0,1.0,prac1.wav,prac2.wav,1.0,None,2,0.0,6.90258,1.0
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,1.0,ws/19.wav,ws/1.wav,2.0,None,1,0.0,0.558629,1.0
3,1.0,ws/22.wav,ws/4.wav,2.0,['2'],2,1.0,1.62566,1.0
4,1.0,ws/1.wav,ws/10.wav,1.0,None,2,0.0,0.117813,1.0


In [52]:
# remove Nan lines in order to make finding type and version less error prone
# we'll do this by removing the rows that have stim[1&&2] set to NaN
sleep_df = sleep_df[pd.notnull(sleep_df['stim1']) & pd.notnull(sleep_df['stim2'])]

sleep_df.head()

,participant,stim1,stim2,correct,key_resp_2.keys,key_resp_9.keys,key_resp_9.corr,key_resp_9.rt,session
0,1.0,prac1.wav,prac2.wav,1.0,None,2,0.0,6.90258,1.0
2,1.0,ws/19.wav,ws/1.wav,2.0,None,1,0.0,0.558629,1.0
3,1.0,ws/22.wav,ws/4.wav,2.0,['2'],2,1.0,1.62566,1.0
4,1.0,ws/1.wav,ws/10.wav,1.0,None,2,0.0,0.117813,1.0
5,1.0,ws/4.wav,ws/13.wav,1.0,['1'],1,1.0,0.425549,1.0


## Adding columns to indicate type, version

In [53]:
# create a dataframe from the key file, try to do a table join SQLkinda thing
# you already know the path to the key
key_path = os.getcwd() + "/key/key.csv"
key_df = pd.read_csv(key_path)

# because we're going to use join, take out the columns you want
key_cols = ['stim1','stim2', 'notes_novword_2', 'TP_novword1','type','version']
key_df = key_df[key_cols]

key_df.head()



,stim1,stim2,notes_novword_2,TP_novword1,type,version
0,ws/1.wav,ws/10.wav,NaN,0.0,non_w,1
1,ws/1.wav,ws/34.wav,non_w_ss,0.0,non_w,1
2,ws/12.wav,ws/3.wav,NaN,0.0,non_w,1
3,ws/3.wav,ws/29.wav,non_w_ss,0.0,non_w,1
4,ws/31.wav,ws/4.wav,non_w_ss,0.0,non_w,1


In [54]:
## merge the two data frames on both the stim_1 and stim_2 values
join_df = pd.merge(sleep_df, key_df, on=['stim1', 'stim2'], how='outer')
join_df.sort_values('participant', inplace=True)
join_df.reset_index(drop=True, inplace=True)

In [55]:
#sanity check

join_df.head(500)

,participant,stim1,stim2,correct,key_resp_2.keys,key_resp_9.keys,key_resp_9.corr,key_resp_9.rt,session,notes_novword_2,TP_novword1,type,version
0,1.0,prac1.wav,prac2.wav,1.0,None,2,0.0,6.90258,1.0,NaN,NaN,NaN,NaN
1,1.0,cat/4-16-1.wav,cat/4-1-16.wav,1.0,NaN,2,0.0,0.460393,1.0,NaN,16.00,three_1nov_1cong,1.0
2,1.0,cat/1-4-13.wav,cat/1-4-16.wav,2.0,NaN,1,0.0,0.694689,1.0,16,13.00,three_2nov_2cong,1.0
3,1.0,cat/11-4-9.wav,cat/21-4-9.wav,1.0,NaN,2,0.0,0.38818,1.0,21,11.00,three_2nov_1cong_1incong,1.0
4,1.0,ws/27.wav,ws/9.wav,2.0,None,1,0.0,0.269042,1.0,NaN,0.50,part_w,1.0
5,1.0,cat/1-14-9.wav,cat/1-10-9.wav,1.0,NaN,1,1.0,0.477008,1.0,10,14.00,three_2nov_2cong,1.0
6,1.0,cat/3-15-9.wav,cat/3-22-9.wav,1.0,NaN,2,0.0,0.542049,1.0,22,15.00,three_2nov_1cong_1incong,1.0
7,1.0,cat/10-4-9.wav,cat/13-4-9.wav,1.0,NaN,1,1.0,0.441135,1.0,13,10.00,three_2nov_2cong,1.0
8,1.0,ws/21.wav,ws/3.wav,2.0,['2'],None,0.0,NaN,1.0,NaN,0.50,part_w,1.0
9,1.0,ws/9.wav,ws/18.wav,1.0,None,1,1.0,0.540822,1.0,NaN,0.00,non_w,1.0


In [56]:
# before we can fix the correct column, we need to fix the mapping, i.e. convert the list vals to their ints
join_df['key_resp_2.keys'] = join_df['key_resp_2.keys'].map({"['1']": 1, "['2']": 2, 1: 1, 2: 2, '1': 1, '2': 2, 'None': 'None'})
join_df['key_resp_9.keys'] = join_df['key_resp_9.keys'].map({"['1']": 1, "['2']": 2, 1: 1, 2: 2, '1': 1, '2': 2, 'None': 'None'})

# for key_resp_9.rt, we need to extract the ints stuck in strings
# a little unstable, but here's what this does
# for all values in key_resp_9.rt, if its a string, assuming that it's a list in a string, extract and cast the value in the list to an int
join_df = join_df.rename(columns = {'key_resp_9.rt':'key_resp_9_rt'})
join_df['key_resp_9_rt'] = join_df.apply( lambda row: ( (float(row['key_resp_9_rt'][1:row['key_resp_9_rt'].find(',')]) if row['key_resp_9_rt'].find(',') > -1
                                                        else float(row['key_resp_9_rt'][1:-1]))
                                                           if isinstance( row['key_resp_9_rt'], str ) == True 
                                                           else row['key_resp_9_rt'] ), axis=1 )

# the correct condition is that each correct must be equal to its corresponding key value, unless its equal to 0
# rename 'key_resp_9.keys' to make indexing easier
join_df = join_df.rename(columns = {'key_resp_9.keys':'key_resp_9'})
join_df.loc[join_df.correct == join_df.key_resp_9, 'key_resp_9.corr'] = 1
join_df.loc[join_df.correct != join_df.key_resp_9, 'key_resp_9.corr'] = 0
join_df.loc[join_df.key_resp_9 == 'None', 'key_resp_9.corr'] = np.NaN

join_df.head(500)

,participant,stim1,stim2,correct,key_resp_2.keys,key_resp_9,key_resp_9.corr,key_resp_9_rt,session,notes_novword_2,TP_novword1,type,version
0,1.0,prac1.wav,prac2.wav,1.0,None,2,0.0,6.902582,1.0,NaN,NaN,NaN,NaN
1,1.0,cat/4-16-1.wav,cat/4-1-16.wav,1.0,NaN,2,0.0,0.460393,1.0,NaN,16.00,three_1nov_1cong,1.0
2,1.0,cat/1-4-13.wav,cat/1-4-16.wav,2.0,NaN,1,0.0,0.694689,1.0,16,13.00,three_2nov_2cong,1.0
3,1.0,cat/11-4-9.wav,cat/21-4-9.wav,1.0,NaN,2,0.0,0.388180,1.0,21,11.00,three_2nov_1cong_1incong,1.0
4,1.0,ws/27.wav,ws/9.wav,2.0,None,1,0.0,0.269042,1.0,NaN,0.50,part_w,1.0
5,1.0,cat/1-14-9.wav,cat/1-10-9.wav,1.0,NaN,1,1.0,0.477008,1.0,10,14.00,three_2nov_2cong,1.0
6,1.0,cat/3-15-9.wav,cat/3-22-9.wav,1.0,NaN,2,0.0,0.542049,1.0,22,15.00,three_2nov_1cong_1incong,1.0
7,1.0,cat/10-4-9.wav,cat/13-4-9.wav,1.0,NaN,1,1.0,0.441135,1.0,13,10.00,three_2nov_2cong,1.0
8,1.0,ws/21.wav,ws/3.wav,2.0,2,None,NaN,NaN,1.0,NaN,0.50,part_w,1.0
9,1.0,ws/9.wav,ws/18.wav,1.0,None,1,1.0,0.540822,1.0,NaN,0.00,non_w,1.0


In [58]:
# Types for creating type_all column
types = {'three_1nov_1cong': 'all_three', 'three_2nov_2cong': 'all_three', 'three_2nov_1cong_1incong': 'all_three',
         'part_w': 'all_word_seg', 'non_w': 'all_word_seg', 'order_TP33': 'order', np.nan: np.nan}

# Create new column where the value for each row depends on the value for column type
join_df['type_all'] = join_df.apply(lambda row: types[row['type']], axis=1)

# DataFrame with type_all column added
join_df.head()


,participant,stim1,stim2,correct,key_resp_2.keys,key_resp_9,key_resp_9.corr,key_resp_9_rt,session,notes_novword_2,TP_novword1,type,version,type_all
0,1.0,prac1.wav,prac2.wav,1.0,None,2,0.0,6.902582,1.0,NaN,NaN,NaN,NaN,NaN
1,1.0,cat/4-16-1.wav,cat/4-1-16.wav,1.0,NaN,2,0.0,0.460393,1.0,NaN,16.0,three_1nov_1cong,1.0,all_three
2,1.0,cat/1-4-13.wav,cat/1-4-16.wav,2.0,NaN,1,0.0,0.694689,1.0,16,13.0,three_2nov_2cong,1.0,all_three
3,1.0,cat/11-4-9.wav,cat/21-4-9.wav,1.0,NaN,2,0.0,0.388180,1.0,21,11.0,three_2nov_1cong_1incong,1.0,all_three
4,1.0,ws/27.wav,ws/9.wav,2.0,None,1,0.0,0.269042,1.0,NaN,0.5,part_w,1.0,all_word_seg


### Output data to csv

In [59]:
join_df.to_csv('output.csv')

# Create pivot table for each session & version combination and merge together into one table

In [60]:
# Dictionary used to create each grouping of columns depending on the session and version
# First tuple represents the sessions and the second tuple represents the versions. 
types = {'all_type': [(1, 2), (1, 2)], 's_all_v_1': [(1, 2), (1,)], 's_all_v_2': [(1, 2), (2,)],
         's_1_v_all': [(1,), (1, 2)], 's_2_v_all': [(2,), (1, 2)], 's_1_v_1': [(1,), (1,)], 's_1_v_2': [(1,), (2,)],
         's_2_v_1': [(2,), (1,)], 's_2_v_2': [(2,), (2,)]}

# Start off with a column of all participants
# Remove any duplicates so that each participant only appears once
sleep_df = join_df['participant'].drop_duplicates().to_frame()

# Table with only a column of participants
sleep_df.head()

,participant
0,1.0
40,2.0
64,3.0
120,4.0
160,101.0


In [61]:
# Go through each session and version combination and create a pivot table for it and merge it with sleep_df DataFrame
for key in types:

    # Only select the rows from the output that match the session and version wanted
    session_version_df = join_df.loc[
        join_df['session'].isin(types[key][0]) & join_df['version'].isin(types[key][1])]

    # Table with rows that satisfy the required sessions and version
    # print session_version_df.head()

    # Create a pivot table using participants as the index, columns as the different types and the values as the mean
    # of the grouped key_resp_9.corr
    type_df = session_version_df.pivot_table(index='participant', columns='type', values='key_resp_9.corr',
                                             aggfunc='mean')

    # column for grand total average for all sessions and versions
    type_df['grand_total'] = session_version_df.groupby(['participant']).mean()['key_resp_9.corr']

    # Rename column names by appending the key (session and version combination) to the end of each column
    for column in type_df.columns.values:
        type_df.rename(columns={column: '%s_%s' % (column, key)}, inplace=True)

    # Add a column for the participants to the pivot table so that it can be merged with sleep_df using participants
    type_df['participant'] = type_df.index.values

    # Pivot Table with column names renamed and participant column added
    # print type_df.head()

    # Merge the pivot table with sleep_df using the participant column
    # The value NaN will be set if there is no data for the participant for the given session and version
    sleep_df = pd.merge(sleep_df, type_df, how='left', on=['participant'])
    
# Table with all session and version combinations
sleep_df.head()

,participant,non_w_s_2_v_1,order_TP33_s_2_v_1,part_w_s_2_v_1,three_1nov_1cong_s_2_v_1,three_2nov_1cong_1incong_s_2_v_1,three_2nov_2cong_s_2_v_1,grand_total_s_2_v_1,non_w_s_all_v_1,order_TP33_s_all_v_1,part_w_s_all_v_1,three_1nov_1cong_s_all_v_1,three_2nov_1cong_1incong_s_all_v_1,three_2nov_2cong_s_all_v_1,grand_total_s_all_v_1,non_w_s_all_v_2,order_TP33_s_all_v_2,part_w_s_all_v_2,three_1nov_1cong_s_all_v_2,three_2nov_1cong_1incong_s_all_v_2,three_2nov_2cong_s_all_v_2,grand_total_s_all_v_2,non_w_s_1_v_2,order_TP33_s_1_v_2,part_w_s_1_v_2,three_1nov_1cong_s_1_v_2,three_2nov_1cong_1incong_s_1_v_2,three_2nov_2cong_s_1_v_2,grand_total_s_1_v_2,non_w_s_1_v_all,order_TP33_s_1_v_all,part_w_s_1_v_all,three_1nov_1cong_s_1_v_all,three_2nov_1cong_1incong_s_1_v_all,three_2nov_2cong_s_1_v_all,grand_total_s_1_v_all,non_w_s_2_v_2,order_TP33_s_2_v_2,part_w_s_2_v_2,three_1nov_1cong_s_2_v_2,three_2nov_1cong_1incong_s_2_v_2,three_2nov_2cong_s_2_v_2,grand_total_s_2_v_2,non_w_all_type,order_TP33_all_type,part_w_all_type,three_1nov_1cong_all_type,three_2nov_1cong_1incong_all_type,three_2nov_2cong_all_type,grand_total_all_type,non_w_s_2_v_all,order_TP33_s_2_v_all,part_w_s_2_v_all,three_1nov_1cong_s_2_v_all,three_2nov_1cong_1incong_s_2_v_all,three_2nov_2cong_s_2_v_all,grand_total_s_2_v_all,non_w_s_1_v_1,order_TP33_s_1_v_1,part_w_s_1_v_1,three_1nov_1cong_s_1_v_1,three_2nov_1cong_1incong_s_1_v_1,three_2nov_2cong_s_1_v_1,grand_total_s_1_v_1
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.500000,0.333333,0.555556,0.500000,0.600000,0.542857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.500000,0.333333,0.555556,0.500000,0.600000,0.542857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.500000,0.333333,0.555556,0.500000,0.600000,0.542857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.500000,0.333333,0.555556,0.500000,0.600000,0.542857
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.5,NaN,NaN,NaN,0.500000,0.5,NaN,0.5,NaN,NaN,NaN,0.5,0.500000,NaN,0.500000,NaN,NaN,NaN,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,NaN,0.500000,NaN,NaN,NaN,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444444,0.166667,0.333333,0.400000,0.571429,0.666667,0.431818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444444,0.166667,0.333333,0.400000,0.571429,0.666667,0.431818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444444,0.166667,0.333333,0.400000,0.571429,0.666667,0.431818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444444,0.166667,0.333333,0.400000,0.571429,0.666667,0.431818
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.666667,0.500000,0.666667,0.166667,0.500000,0.526316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.666667,0.500000,0.666667,0.166667,0.500000,0.526316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.666667,0.500000,0.666667,0.166667,0.500000,0.526316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.666667,0.500000,0.666667,0.166667,0.500000,0.526316
4,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375000,0.666667,0.250000,0.333333,0.600000,0.500000,0.447368,0.4,0.583333,1.0,0.888889,0.666667,0.666667,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375000,0.666667,0.250000,0.333333,0.600000,0.500000,0.447368,0.4,0.583333,1.0,0.888889,0.666667,0.666667,0.666667,0.388889,0.611111,0.666667,0.611111,0.636364,0.583333,0.569767,0.4,0.583333,1.0,0.888889,0.666667,0.666667,0.666667,0.375000,0.666667,0.250000,0.333333,0.600000,0.500000,0.447368


# Read participants CSV and combine with above table

In [62]:
# Load the participant CSV file into Pandas
participant_df = pd.read_csv("participants.csv")

participant_df.head()

,participant,age_group,condition
0,101,1,QW
1,102,1,QW
2,103,1,QW
3,104,1,QW
4,105,1,QW


In [63]:
# Do a join on 'participant' for the 2 tables
new_output = pd.merge(participant_df, sleep_df, on="participant")

new_output.head()

,participant,age_group,condition,non_w_s_2_v_1,order_TP33_s_2_v_1,part_w_s_2_v_1,three_1nov_1cong_s_2_v_1,three_2nov_1cong_1incong_s_2_v_1,three_2nov_2cong_s_2_v_1,grand_total_s_2_v_1,non_w_s_all_v_1,order_TP33_s_all_v_1,part_w_s_all_v_1,three_1nov_1cong_s_all_v_1,three_2nov_1cong_1incong_s_all_v_1,three_2nov_2cong_s_all_v_1,grand_total_s_all_v_1,non_w_s_all_v_2,order_TP33_s_all_v_2,part_w_s_all_v_2,three_1nov_1cong_s_all_v_2,three_2nov_1cong_1incong_s_all_v_2,three_2nov_2cong_s_all_v_2,grand_total_s_all_v_2,non_w_s_1_v_2,order_TP33_s_1_v_2,part_w_s_1_v_2,three_1nov_1cong_s_1_v_2,three_2nov_1cong_1incong_s_1_v_2,three_2nov_2cong_s_1_v_2,grand_total_s_1_v_2,non_w_s_1_v_all,order_TP33_s_1_v_all,part_w_s_1_v_all,three_1nov_1cong_s_1_v_all,three_2nov_1cong_1incong_s_1_v_all,three_2nov_2cong_s_1_v_all,grand_total_s_1_v_all,non_w_s_2_v_2,order_TP33_s_2_v_2,part_w_s_2_v_2,three_1nov_1cong_s_2_v_2,three_2nov_1cong_1incong_s_2_v_2,three_2nov_2cong_s_2_v_2,grand_total_s_2_v_2,non_w_all_type,order_TP33_all_type,part_w_all_type,three_1nov_1cong_all_type,three_2nov_1cong_1incong_all_type,three_2nov_2cong_all_type,grand_total_all_type,non_w_s_2_v_all,order_TP33_s_2_v_all,part_w_s_2_v_all,three_1nov_1cong_s_2_v_all,three_2nov_1cong_1incong_s_2_v_all,three_2nov_2cong_s_2_v_all,grand_total_s_2_v_all,non_w_s_1_v_1,order_TP33_s_1_v_1,part_w_s_1_v_1,three_1nov_1cong_s_1_v_1,three_2nov_1cong_1incong_s_1_v_1,three_2nov_2cong_s_1_v_1,grand_total_s_1_v_1
0,101,1,QW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375,0.666667,0.25,0.333333,0.600000,0.500000,0.447368,0.4,0.583333,1.0,0.888889,0.666667,0.666667,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375,0.666667,0.25,0.333333,0.600000,0.500000,0.447368,0.4,0.583333,1.0,0.888889,0.666667,0.666667,0.666667,0.388889,0.611111,0.666667,0.611111,0.636364,0.583333,0.569767,0.40,0.583333,1.00,0.888889,0.666667,0.666667,0.666667,0.375,0.666667,0.25,0.333333,0.6,0.5,0.447368
1,102,1,QW,0.75,0.333333,0.75,1.000000,0.500000,0.333333,0.641026,0.750,0.333333,0.75,1.000000,0.500000,0.333333,0.641026,0.6,0.666667,0.5,0.555556,0.500000,0.833333,0.617021,0.6,0.666667,0.5,0.555556,0.500000,0.833333,0.617021,0.600,0.666667,0.50,0.555556,0.500000,0.833333,0.617021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.555556,0.625000,0.777778,0.500000,0.583333,0.627907,0.75,0.333333,0.75,1.000000,0.500000,0.333333,0.641026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103,1,QW,0.25,0.500000,0.50,0.555556,0.666667,0.500000,0.487179,0.250,0.500000,0.50,0.555556,0.666667,0.500000,0.487179,0.6,0.416667,0.6,0.333333,0.833333,0.666667,0.541667,0.6,0.416667,0.6,0.333333,0.833333,0.666667,0.541667,0.600,0.416667,0.60,0.333333,0.833333,0.666667,0.541667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444444,0.444444,0.555556,0.444444,0.750000,0.583333,0.517241,0.25,0.500000,0.50,0.555556,0.666667,0.500000,0.487179,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,104,1,QW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,0.416667,0.6,0.333333,0.666667,0.833333,0.479167,0.3,0.416667,0.6,0.333333,0.666667,0.833333,0.479167,0.300,0.416667,0.60,0.333333,0.666667,0.833333,0.479167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300000,0.416667,0.600000,0.333333,0.666667,0.833333,0.479167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105,1,QW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875,0.333333,1.00,0.666667,0.500000,0.500000,0.641026,0.4,0.583333,0.6,0.444444,0.666667,0.833333,0.562500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875,0.333333,1.00,0.666667,0.500000,0.500000,0.641026,0.4,0.583333,0.6,0.444444,0.666667,0.833333,0.562500,0.611111,0.500000,0.777778,0.555556,0.583333,0.666667,0.597701,0.40,0.583333,0.60,0.444444,0.666667,0.833333,0.562500,0.875,0.333333,1.00,0.666667,0.5,0.5,0.641026


# Output data to CSV

In [64]:
new_output.to_csv('summary.csv')